In [2]:
import pandas as pd
df = pd.read_csv("/work/pi_wenlongzhao_umass_edu/27/janet/query_expansion/data/CoSQA_explanations_query_code.csv")
df.head()

,Unnamed: 0,query_id,doc,corpus_id,code,explanation_granite_1,explanation_granite_2,explanation_granite_3,explanation_granite_4,explanation_deepseek_1_cleaned,explanation_deepseek_2_cleaned,explanation_deepseek_3_cleaned,explanation_deepseek_4_cleaned
0,0,q1,python code to write bool value 1,d1,"def writeBoolean(self, n):\n """"""\n ...",\nThe provided code is a Python method named `...,"\n1. `def writeBoolean(self, n):` - This line ...",\nThis code snippet is a function that writes ...,\nThe code snippet provided is a method named ...,The doc string and code both write the boolean...,"Alright, I need to break down this Python code...",The code snippet writes a boolean value to a s...,The code snippet defines a method called write...
1,1,q2,"""python how to manipulate clipboard""",d2,"def paste(xsel=False):\n """"""Returns system ...",\nThe provided Python code snippet defines a f...,\n1. `def paste(xsel=False):` - This line defi...,\nThis code snippet is a function in Python th...,\nThe code snippet provided is a Python functi...,The paste function pastes data from the clipbo...,The code snippet begins by defining a function...,The code snippet is a function called paste th...,The code snippet defines a function called pas...
2,2,q3,python colored output to html,d3,"def _format_json(data, theme):\n """"""Pretty ...","\nThis Python function, `_format_json`, takes ...","\n1. `def _format_json(data, theme):` - This l...","\nThis code snippet, named `_format_json`, is ...",\n```python\nimport json\nimport sys\nimport p...,The docstring explains that the function _form...,"```pythondef _format_json(data, theme): """"""...",This code snippet is a function called _format...,To explain the code snippet in a way that can ...
3,3,q4,"python ""create directory"" using ""relative path""",d4,"def create_path(path):\n """"""Creates a absol...","\nThe provided Python function, `create_path(p...",\n1. The code snippet defines a function named...,\nThis code snippet helps you create a new dir...,\nThe provided code snippet is a Python functi...,The doc and code create an absolute path from ...,"Okay, I need to explain this Python code snipp...",This code creates a file system where any path...,The code snippet defines a function called cre...
4,4,q5,python column of an array,d5,"def _vector_or_scalar(x, type='row'):\n """"""...",\nThe provided Python code defines a function ...,\nThis code snippet defines a function called ...,\nThis code snippet is a function that convert...,\nThe code snippet provided is a function defi...,The code checks if an object `x` is an array o...,"Alright, so I've got this Python code snippet ...",The code snippet is a Python function called _...,"Okay, I'm going to try to figure out how to ex..."


In [3]:
import os
import nltk
import pandas as pd
import itertools
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# === Set path to your manually extracted nltk_data ===
nltk_data_path = os.path.join(os.getcwd(), 'nltk_data')  # Change if needed
nltk.data.path.append(nltk_data_path)

# === Load your CSV ===
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# === NLP Functions ===
def get_synonyms(word):
    return {lemma.name().replace("_", " ") for syn in wordnet.synsets(word) for lemma in syn.lemmas()}

def get_related_terms(word):
    related = set()
    for syn in wordnet.synsets(word):
        for hyper in syn.hypernyms():
            related |= {lemma.name().replace("_", " ") for lemma in hyper.lemmas()}
        for hypo in syn.hyponyms():
            related |= {lemma.name().replace("_", " ") for lemma in hypo.lemmas()}
    return related

def get_normal_forms(word):
    return {stemmer.stem(word), lemmatizer.lemmatize(word)}

def get_query_permutations(query):
    words = query.split()
    return {' '.join(p) for p in set(itertools.permutations(words)) if p != tuple(words)}

def expand_query(query, max_syns=5, max_rel=5, use_permutations=False):
    query = query.lower()
    words = query.split()
    expanded_queries = set([query])

    for word in words:
        # Limit synonyms and related terms for speed
        synonyms = list(get_synonyms(word))[:max_syns]
        related = list(get_related_terms(word))[:max_rel]
        normal_forms = get_normal_forms(word)

        for term in set(synonyms + related + list(normal_forms)):
            if term != word:
                new_query = query.replace(word, term)
                expanded_queries.add(new_query)

    if use_permutations and len(words) <= 5:
        expanded_queries |= get_query_permutations(query)

    return list(expanded_queries)


def rank_queries(expanded_queries, original_query, top_k=10):
    if len(expanded_queries) > 2000:
        expanded_queries = expanded_queries[:2000]  # cap for safety

    corpus = [original_query] + expanded_queries
    tfidf = TfidfVectorizer().fit_transform(corpus)
    orig_vec = tfidf[0]
    similarity_scores = cosine_similarity(orig_vec, tfidf[1:])[0]
    ranked = sorted(zip(expanded_queries, similarity_scores), key=lambda x: x[1], reverse=True)
    return ranked[:top_k]


# === Main ===
if __name__ == "__main__":
    query = df.iloc[0]["doc"]  # You can loop through all rows if needed
    try:
        top_k = int(input("How many top results do you want? (Top K): ").strip())
    except ValueError:
        top_k = 10

    expanded = expand_query(query)
    ranked_expansions = rank_queries(expanded, query, top_k=top_k)

    print(f"\n🔍 Original Query: {query}")
    print(f"\n✨ Top {top_k} Expanded Queries (ranked by semantic similarity):")
    for i, (exp, score) in enumerate(ranked_expansions, 1):
        print(f"{i:2d}. {exp} (similarity: {score:.4f})")


In [4]:
import zipfile
import os

zip_path = os.path.expanduser('nltk_data/corpora/omw-1.4.zip')
extract_to = os.path.expanduser('nltk_data/corpora/omw-1.4')

# Create target folder if it doesn't exist
os.makedirs(extract_to, exist_ok=True)

# Extract manually
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("✅ Extracted to:", extract_to)


✅ Extracted to: nltk_data/corpora/omw-1.4


In [6]:
import os
import nltk

# Set and create your custom nltk_data path
nltk_data_path = os.path.join(os.getcwd(), 'nltk_data')
os.makedirs(nltk_data_path, exist_ok=True)
nltk.data.path.append(nltk_data_path)

# 🔁 Force download of punkt again to your path
nltk.download('punkt', download_dir=nltk_data_path)
nltk.download('stopwords', download_dir=nltk_data_path)
nltk.download('wordnet', download_dir=nltk_data_path)


[nltk_data] Downloading package punkt to /work/pi_wenlongzhao_umass_ed
[nltk_data]     u/27/janet/query_expansion/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /work/pi_wenlongzhao_umas
[nltk_data]     s_edu/27/janet/query_expansion/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /work/pi_wenlongzhao_umass_
[nltk_data]     edu/27/janet/query_expansion/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
import nltk
import os

nltk_data_path = os.path.join(os.getcwd(), 'nltk_data')
nltk.data.path.clear()  # Clear any old paths
nltk.data.path.append(nltk_data_path)


In [4]:
import nltk
import os

nltk_data_path = os.path.join(os.getcwd(), 'nltk_data')
os.makedirs(nltk_data_path, exist_ok=True)
nltk.data.path.clear()
nltk.data.path.append(nltk_data_path)

# Re-download WordNet
nltk.download('wordnet', download_dir=nltk_data_path)


[nltk_data] Downloading package wordnet to /work/pi_wenlongzhao_umass_
[nltk_data]     edu/27/janet/query_expansion/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
import nltk
from nltk.tokenize import word_tokenize
nltk.data.path = ["nltk_data"]  # ✅ Point to your custom path

print(word_tokenize("This is a test sentence."))


['This', 'is', 'a', 'test', 'sentence', '.']


In [6]:
from nltk.corpus import wordnet as wn
for syn in wn.synsets("tweet"): 
    print ("{0}:{1}:\t\t\t{2}".format(syn.name(), syn.lexname(), syn.definition()))

tweet.n.01:noun.event:			a weak chirping sound as of a small bird
tweet.v.01:verb.perception:			make a weak, chirping sound
pinch.v.01:verb.contact:			squeeze tightly between the fingers


In [9]:
import os
import string
import nltk
import pandas as pd
from nltk.corpus import wordnet, stopwords
from nltk.tokenize import word_tokenize

# === Set custom NLTK data path ===
nltk_data_path = os.path.join(os.getcwd(), 'nltk_data')
os.makedirs(nltk_data_path, exist_ok=True)
nltk.data.path.append(nltk_data_path)

# === Download necessary resources to that path ===
# nltk.download('punkt', download_dir=nltk_data_path)
# nltk.download('stopwords', download_dir=nltk_data_path)
# nltk.download('wordnet', download_dir=nltk_data_path)

# === Load stopwords ===
stop_words = set(stopwords.words("english"))

# === Load input CSV ===
input_path = "/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CoSQA_explanations_query_code.csv"  # Update this if needed
df = pd.read_csv(input_path).iloc[:2]

# === Function to get WordNet synonyms (limit 3 per word) ===
def get_synonyms(word, limit=3):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            name = lemma.name().replace('_', ' ')
            if name != word:
                synonyms.add(name)
            if len(synonyms) >= limit:
                return list(synonyms)
    return list(synonyms)

# === Expand queries ===
expanded_data = []

for _, row in df.iterrows():
    query_id = row['query_id']
    query = row['doc'].lower().translate(str.maketrans('', '', string.punctuation))
    words = word_tokenize(query)
    words = [w for w in words if w not in stop_words]

    synonyms = []
    for word in words:
        synonyms += get_synonyms(word, limit=3)

    expanded_data.append({
        "id": query_id,
        "original_query": row['doc'],
        "expanded_query": ' '.join(synonyms)
    })

# === Save to output CSV ===
output_path = "/work/pi_wenlongzhao_umass_edu/27/janet/query_expansion/results/queries_expanded_nltk.csv"
pd.DataFrame(expanded_data).to_csv(output_path, index=False)
print(f"✅ Expanded queries saved to {output_path}")


✅ Expanded queries saved to /work/pi_wenlongzhao_umass_edu/27/janet/query_expansion/results/queries_expanded_nltk.csv


# LangChain

In [1]:
import os
import pandas as pd
from langchain.chat_models import ChatOpenAI
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from langchain.schema import SystemMessage, HumanMessage


# === LangChain + DeepSeek ===
llm = ChatOpenAI(
    model_name="/datasets/ai/deepseek/hub/models--deepseek-ai--DeepSeek-R1-Distill-Qwen-1.5B/snapshots/530ca3e1ad39d440e182c2e4317aa40f012512fa",
    openai_api_base="http://localhost:8000/v1",  # Unity-hosted endpoint
    openai_api_key="dummy-key",  # required but not used by vLLM
    temperature=0.5
)
def clean_lines(lines):
    return [
        line.strip().lstrip("0123456789).:- ").strip('" ')
        for line in lines if line.strip()
    ]

def expand_query_with_deepseek(query, llm, num_expansions=5):
    # prompt = (
    #     f"Expand the following programming-related query into {num_expansions} semantically similar variants. "
    #     f"Keep technical keywords like 'python', 'list', 'json' unchanged.\n\n"
    #     f"Original query: {query}\n\n"
    #     f"Expanded queries:"
    # )
    # response = llm.invoke(prompt)
    # lines = response.content.strip().split("\n")
    # expansions = [line.strip("1234567890. ").strip() for line in lines if line.strip()]
    # system_message = SystemMessage(content=(
    # "You are a query expansion engine for programming questions. "
    # "Only return a clean list of semantically similar queries. "
    # "Do not explain, do not chat. "
    # "Preserve technical terms such as 'python', 'list', 'json', etc."
    # ))

    # user_prompt = (
    #     f"Expand the following programming-related query into 10 semantically similar variants. "
    #     f"Keep technical keywords like 'python', 'list', 'json' unchanged.\n\n"
    #     f"Original query: {query}\n\n"
    #     f"Expanded queries:"
    # )

    # response = llm([system_message, HumanMessage(content=user_prompt)])
    # lines = response.content.splitlines()
    # expansions = [line.strip("1234567890. ").strip() for line in lines if line.strip()]
    # return expansions
    system_msg = SystemMessage(content=(
        "You are a query expansion engine specialized in programming queries. "
        "Return a list of clean, short, semantically similar variants of the input query. "
        "Do not include any explanation, preambles, or list numbers. "
        "Keep programming terms like 'python', 'list', 'json', etc. unchanged. "
        "Each expansion must be a standalone query."
    ))

    user_msg = HumanMessage(content=(
        f"Expand this query into {num_expansions} semantically similar variants:\n"
        f"{query}\n\n"
        "Only return the expanded queries, one per line, no numbering or bullet points."
    ))

    try:
        response = llm([system_msg, user_msg])
        expanded = clean_lines(response.content.splitlines())
        return expanded
    except Exception as e:
        print(f"Error expanding query '{query}': {e}")
        return []

def rank_queries(expanded_queries, original_query, top_k=5):
    corpus = [original_query] + expanded_queries
    tfidf = TfidfVectorizer().fit_transform(corpus)
    sims = cosine_similarity(tfidf[0:1], tfidf[1:])[0]
    ranked = sorted(zip(expanded_queries, sims), key=lambda x: x[1], reverse=True)
    return ranked[:top_k]

# === File paths ===
input_csv = "/work/pi_wenlongzhao_umass_edu/27/janet/query_expansion/data/CoSQA_explanations_query_code.csv"
output_csv = "/work/pi_wenlongzhao_umass_edu/27/janet/query_expansion/results/code_queries_deepseek_expanded_10.csv"

df = pd.read_csv(input_csv).iloc[:10]
results = []

# === Main Loop ===
for _, row in df.iterrows():
    qid = row['query_id']
    query = row['doc']

    try:
        expanded = expand_query_with_deepseek(query, llm, num_expansions=10)
        ranked = rank_queries(expanded, query, top_k=5)

        for rank, (exp_q, score) in enumerate(ranked, 1):
            results.append({
                "id": qid,
                "original_query": query,
                "rank": rank,
                "expanded_query": exp_q,
                "similarity": round(score, 4)
            })
    except Exception as e:
        print(f"⚠️ Error expanding query ID {qid}: {e}")

# === Save to CSV ===
pd.DataFrame(results).to_csv(output_csv, index=False)
print(f"Done! Results saved to {output_csv}")


/tmp/ipykernel_1606488/1111135649.py:10: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(
/tmp/ipykernel_1606488/1111135649.py:65: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm([system_msg, user_msg])


Done! Results saved to /work/pi_wenlongzhao_umass_edu/27/janet/query_expansion/results/code_queries_deepseek_expanded_10.csv


In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage

# === Connect to local DeepSeek model via OpenAI-compatible vLLM server ===
llm = ChatOpenAI(
    model_name="/datasets/ai/deepseek/hub/models--deepseek-ai--DeepSeek-R1-Distill-Qwen-1.5B/snapshots/530ca3e1ad39d440e182c2e4317aa40f012512fa",
    openai_api_base="http://localhost:8000/v1",
    openai_api_key="dummy-key",  # required by LangChain
    temperature=0.0
)

# === Define a strict system instruction ===
system_prompt = SystemMessage(content=(
    "You are a query expansion engine. "
    "ONLY return clean, short, semantically similar variants of the given programming query. "
    "NO explanation. NO chat. NO thoughts. "
    "Return each expanded query on its own line. No bullets, no numbers, no quotes."
))

# === Define user query ===
query = "python code to write bool value 1"
user_prompt = HumanMessage(content=f"Expand this query into 10 variants:\n{query}")

# === Run the model ===
response = llm([system_prompt, user_prompt])

# === Process result ===
expanded_queries = [line.strip().lstrip("0123456789).:- ").strip('" ')
                    for line in response.content.splitlines()
                    if line.strip() and "let me" not in line.lower()]

# === Print results ===
print("\n🔍 Original Query:")
print(query)
print("\n✨ Expanded Variants:")
for i, q in enumerate(expanded_queries, 1):
    print(f"{i}. {q}")



🔍 Original Query:
python code to write bool value 1

✨ Expanded Variants:
1. First, I should understand what the original query is asking. It seems like someone wants a Python code snippet that creates a boolean value of 1. So, the goal is to generate code that evaluates to True when 1 is assigned to a boolean variable.
2. I should consider different ways to structure the code. Maybe using if statements, elifs, else blocks, or even try-except for error handling. Also, using logical operators like and, or, not, or boolean functions like bool(), int(), etc.
3. I should also think about variable names. It's better to use descriptive names like bool_value or bool_val. Maybe even include comments to explain what the code does.
4. Another thought: perhaps the user wants to test the code or use it in a larger program. So, including examples or explanations of how to use the code would be helpful.
5. I should also consider different programming paradigms, like object-oriented or functional ap

In [2]:
# import os
# import pandas as pd
# from langchain.chat_models import ChatOpenAI
# from langchain.schema import SystemMessage, HumanMessage
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity

# # === Config ===
# MODEL_NAME="/datasets/ai/deepseek/hub/models--deepseek-ai--DeepSeek-R1-Distill-Qwen-1.5B/snapshots/530ca3e1ad39d440e182c2e4317aa40f012512fa"
# API_BASE = "http://localhost:8000/v1"  # change port if needed
# API_KEY = "dummy-key"
# INPUT_CSV = "/work/pi_wenlongzhao_umass_edu/27/janet/query_expansion/data/CoSQA_explanations_query_code.csv"
# OUTPUT_CSV = "/work/pi_wenlongzhao_umass_edu/27/janet/query_expansion/results/code_queries_deepseek_expanded_clean.csv"
# TOP_K = 5
# NUM_EXPANSIONS = 10

# # === Initialize LangChain LLM ===
# llm = ChatOpenAI(
#     model_name=MODEL_NAME,
#     openai_api_base=API_BASE,
#     openai_api_key=API_KEY,
#     temperature=0.0
# )

# # === Clean lines (basic only) ===
# def clean_lines_basic(lines):
#     return [
#         line.strip().lstrip("0123456789).:- ").strip('" ')
#         for line in lines if line.strip()
#     ]

# def expand_query_llm(query, num_expansions=10):
#     system_msg = SystemMessage(content=(
#         "You are a query expansion engine specialized in programming queries. "
#         "Return a list of semantically similar variants of the input query. "
#         "Only return one query per line. No explanations or list formatting."
#     ))

#     user_msg = HumanMessage(content=(
#         f"Expand this query into {num_expansions} semantically similar variants:\n"
#         f"{query}\n\n"
#         "Only return the expanded queries, one per line."
#     ))

#     try:
#         response = llm([system_msg, user_msg])
#         return clean_lines_basic(response.content.splitlines())
#     except Exception as e:
#         print(f"❌ Error expanding query '{query}': {e}")
#         return []

# def rank_queries(expanded_queries, original_query, top_k=5):
#     corpus = [original_query] + expanded_queries
#     tfidf = TfidfVectorizer().fit_transform(corpus)
#     orig_vec = tfidf[0]
#     sims = cosine_similarity(orig_vec, tfidf[1:])[0]
#     ranked = sorted(zip(expanded_queries, sims), key=lambda x: x[1], reverse=True)
#     return ranked[:top_k]

# # === Run Query Expansion ===
# df = pd.read_csv(INPUT_CSV).iloc[:1]
# results = []

# for _, row in df.iterrows():
#     query_id = row['query_id']
#     query = row['doc']
#     expansions = expand_query_llm(query, num_expansions=NUM_EXPANSIONS)

#     if not expansions:
#         continue

#     ranked = rank_queries(expansions, query, top_k=TOP_K)
#     for rank, (q, score) in enumerate(ranked, 1):
#         results.append({
#             "id": query_id,
#             "original_query": query,
#             "rank": rank,
#             "expanded_query": q,
#             "similarity": round(score, 4)
#         })

# # Save to CSV
# pd.DataFrame(results).to_csv(OUTPUT_CSV, index=False)
# print(f"✅ Saved top {TOP_K} expanded queries per input to: {OUTPUT_CSV}")
